In [109]:
import numpy as np 
import pandas as pd 

import nltk
# nltk.download('punkt')
from nltk.corpus import stopwords
import re
from collections import Counter

from tqdm import tqdm
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


# Load Data

In [19]:
PATH_TRAIN_DATA = '../data/train.tsv'
# PATH_TEST_DATA = '../data/test.tsv'

train_data = pd.read_csv(PATH_TRAIN_DATA, sep = '\t', encoding='utf-8')
train_data.head()

,title,content
0,"Singer, activist Joan Baez becomes Kennedy Cen...",Kennedy Center Honoree Joan Baez reflects on 6...
1,'Nothing but problems': Shipwreck tear-down en...,Demolition of a large cargo ship along the coa...
2,Report: At least 13 dead in Istanbul bombings,Report : At least 13 dead in Istanbul bombings...
3,Senate Republicans Pressure Joe Biden to Withd...,Several congressional Republicans are expressi...
4,The Unwelcome Return of the Real Purveyors of ...,With the mainstream media still obsessing abou...


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89928 entries, 0 to 89927
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    89923 non-null  object
 1   content  89800 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [7]:
train_data.isna().sum()

title        5
content    128
dtype: int64

In [20]:
# drop nulls
train_data = train_data.dropna().reset_index(drop=True)
print(train_data.shape)
train_data.head()

(89795, 2)


,title,content
0,"Singer, activist Joan Baez becomes Kennedy Cen...",Kennedy Center Honoree Joan Baez reflects on 6...
1,'Nothing but problems': Shipwreck tear-down en...,Demolition of a large cargo ship along the coa...
2,Report: At least 13 dead in Istanbul bombings,Report : At least 13 dead in Istanbul bombings...
3,Senate Republicans Pressure Joe Biden to Withd...,Several congressional Republicans are expressi...
4,The Unwelcome Return of the Real Purveyors of ...,With the mainstream media still obsessing abou...


# sampling


In [21]:
sample_data = train_data.sample(frac = 0.1, random_state=42).reset_index(drop=True)
sample_data

,title,content
0,Violence escalates in Mexico Protests,There 's more violence in the ongoing protests...
1,Olympics: Italy and Jamaica take sprint relay ...,The Olympics have always been a chance for nat...
2,"Facebook Blacklists 78,000+ Users for Violatin...",Facebook has reportedly blacklisted more than ...
3,Exclusive: Blackburn Demands Answers on Biden'...,Sen. Marsha Blackburn ( R-TN ) and Rep. Chuck ...
4,14-year-old boy shot in West Pullman home,A 14-year-old boy was shot in his home early T...
...,...,...
8975,Poll: 56 Percent of GOP Voters Oppose Earmarks,"According to Morning Consult/Politico poll , m..."
8976,Chicago Fire Department needs stronger policie...,"“ Women are treated like garbage .\nPeriod , ”..."
8977,South Africa’s unrest and the ANC’s many failings,The ANC has failed to dismantle the oppressive...
8978,'I didn't like it at all' - Rangnick unhappy d...,Last updated on .From the section Premier Leag...


In [29]:
# guardamos
path_to_save = '../data/sample/sample_data.csv'
sample_data.to_csv(path_to_save, sep = '\t', index=False)

In [31]:
# sample_data = pd.read_csv(path_to_save, sep = '\t')
# sample_data

In [33]:
i = 0
print(sample_data.iloc[i]['title'])
print("")
print(sample_data.iloc[i]['content'])

Violence escalates in Mexico  Protests

There 's more violence in the ongoing protests over the disappearance and apparent murder of 43 college students in Mexico @ @ @ @ @ public safety in the Mexican state of Guerrero was temporarily detained by protesters .
CBS @ @ @ @ @ CBSN by phone from where the kidnapping happened -- with the latest .


# Tecnicas clasicas

preprocesado de tecnicas clasicas

In [32]:
sample_path = '../data/sample/sample_data.csv'
sample_data = pd.read_csv(path_to_save, sep = '\t')
sample_data

,title,content
0,Violence escalates in Mexico Protests,There 's more violence in the ongoing protests...
1,Olympics: Italy and Jamaica take sprint relay ...,The Olympics have always been a chance for nat...
2,"Facebook Blacklists 78,000+ Users for Violatin...",Facebook has reportedly blacklisted more than ...
3,Exclusive: Blackburn Demands Answers on Biden'...,Sen. Marsha Blackburn ( R-TN ) and Rep. Chuck ...
4,14-year-old boy shot in West Pullman home,A 14-year-old boy was shot in his home early T...
...,...,...
8975,Poll: 56 Percent of GOP Voters Oppose Earmarks,"According to Morning Consult/Politico poll , m..."
8976,Chicago Fire Department needs stronger policie...,"“ Women are treated like garbage .\nPeriod , ”..."
8977,South Africa’s unrest and the ANC’s many failings,The ANC has failed to dismantle the oppressive...
8978,'I didn't like it at all' - Rangnick unhappy d...,Last updated on .From the section Premier Leag...


In [45]:
sample_data['text_joined'] = sample_data.apply(lambda x: f"{x['title']}\n {x['content']}", axis=1)
sample_data

,title,content,text_joined
0,Violence escalates in Mexico Protests,There 's more violence in the ongoing protests...,Violence escalates in Mexico Protests\n There...
1,Olympics: Italy and Jamaica take sprint relay ...,The Olympics have always been a chance for nat...,Olympics: Italy and Jamaica take sprint relay ...
2,"Facebook Blacklists 78,000+ Users for Violatin...",Facebook has reportedly blacklisted more than ...,"Facebook Blacklists 78,000+ Users for Violatin..."
3,Exclusive: Blackburn Demands Answers on Biden'...,Sen. Marsha Blackburn ( R-TN ) and Rep. Chuck ...,Exclusive: Blackburn Demands Answers on Biden'...
4,14-year-old boy shot in West Pullman home,A 14-year-old boy was shot in his home early T...,14-year-old boy shot in West Pullman home\n A ...
...,...,...,...
8975,Poll: 56 Percent of GOP Voters Oppose Earmarks,"According to Morning Consult/Politico poll , m...",Poll: 56 Percent of GOP Voters Oppose Earmarks...
8976,Chicago Fire Department needs stronger policie...,"“ Women are treated like garbage .\nPeriod , ”...",Chicago Fire Department needs stronger policie...
8977,South Africa’s unrest and the ANC’s many failings,The ANC has failed to dismantle the oppressive...,South Africa’s unrest and the ANC’s many faili...
8978,'I didn't like it at all' - Rangnick unhappy d...,Last updated on .From the section Premier Leag...,'I didn't like it at all' - Rangnick unhappy d...


In [47]:
# limpieza

def preprocess_text(text: str) -> str:

    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text)

    # remove stopwords
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if not w.lower() in stopwords.words("english")]
    text = " ".join(tokens)
    text = text.lower().strip()

    return text

In [48]:
preprocess_text(sample_data.iloc[i]['text_joined'])

'violence escalates mexico protests violence ongoing protests disappearance apparent murder college students mexico public safety mexican state guerrero temporarily detained protesters cbs cbsn phone kidnapping happened latest'

In [50]:
sample_data['text_cleaned'] = sample_data['text_joined'].apply(lambda text: preprocess_text(text))
sample_data = sample_data[sample_data['text_cleaned'] != '']

In [52]:
sample_data = sample_data.reset_index(drop=True)
sample_data

,title,content,text_joined,text_cleaned
0,Violence escalates in Mexico Protests,There 's more violence in the ongoing protests...,Violence escalates in Mexico Protests\n There...,violence escalates mexico protests violence on...
1,Olympics: Italy and Jamaica take sprint relay ...,The Olympics have always been a chance for nat...,Olympics: Italy and Jamaica take sprint relay ...,olympics italy jamaica take sprint relay golds...
2,"Facebook Blacklists 78,000+ Users for Violatin...",Facebook has reportedly blacklisted more than ...,"Facebook Blacklists 78,000+ Users for Violatin...",facebook blacklists users violating qanon poli...
3,Exclusive: Blackburn Demands Answers on Biden'...,Sen. Marsha Blackburn ( R-TN ) and Rep. Chuck ...,Exclusive: Blackburn Demands Answers on Biden'...,exclusive blackburn demands answers biden flyi...
4,14-year-old boy shot in West Pullman home,A 14-year-old boy was shot in his home early T...,14-year-old boy shot in West Pullman home\n A ...,year old boy shot west pullman home year old b...
...,...,...,...,...
8975,Poll: 56 Percent of GOP Voters Oppose Earmarks,"According to Morning Consult/Politico poll , m...",Poll: 56 Percent of GOP Voters Oppose Earmarks...,poll percent gop voters oppose earmarks accord...
8976,Chicago Fire Department needs stronger policie...,"“ Women are treated like garbage .\nPeriod , ”...",Chicago Fire Department needs stronger policie...,chicago fire department needs stronger policie...
8977,South Africa’s unrest and the ANC’s many failings,The ANC has failed to dismantle the oppressive...,South Africa’s unrest and the ANC’s many faili...,south africa unrest anc many failings anc fail...
8978,'I didn't like it at all' - Rangnick unhappy d...,Last updated on .From the section Premier Leag...,'I didn't like it at all' - Rangnick unhappy d...,like rangnick unhappy despite cavani earning p...


In [53]:
clean_path = '../data/sample/clean_sample_data.csv'
sample_data.to_csv(clean_path, sep= '\t', index=False)

In [54]:
clean_path = '../data/sample/clean_sample_data.csv'
sample_data = pd.read_csv(clean_path, sep= '\t')
sample_data

,title,content,text_joined,text_cleaned
0,Violence escalates in Mexico Protests,There 's more violence in the ongoing protests...,Violence escalates in Mexico Protests\n There...,violence escalates mexico protests violence on...
1,Olympics: Italy and Jamaica take sprint relay ...,The Olympics have always been a chance for nat...,Olympics: Italy and Jamaica take sprint relay ...,olympics italy jamaica take sprint relay golds...
2,"Facebook Blacklists 78,000+ Users for Violatin...",Facebook has reportedly blacklisted more than ...,"Facebook Blacklists 78,000+ Users for Violatin...",facebook blacklists users violating qanon poli...
3,Exclusive: Blackburn Demands Answers on Biden'...,Sen. Marsha Blackburn ( R-TN ) and Rep. Chuck ...,Exclusive: Blackburn Demands Answers on Biden'...,exclusive blackburn demands answers biden flyi...
4,14-year-old boy shot in West Pullman home,A 14-year-old boy was shot in his home early T...,14-year-old boy shot in West Pullman home\n A ...,year old boy shot west pullman home year old b...
...,...,...,...,...
8975,Poll: 56 Percent of GOP Voters Oppose Earmarks,"According to Morning Consult/Politico poll , m...",Poll: 56 Percent of GOP Voters Oppose Earmarks...,poll percent gop voters oppose earmarks accord...
8976,Chicago Fire Department needs stronger policie...,"“ Women are treated like garbage .\nPeriod , ”...",Chicago Fire Department needs stronger policie...,chicago fire department needs stronger policie...
8977,South Africa’s unrest and the ANC’s many failings,The ANC has failed to dismantle the oppressive...,South Africa’s unrest and the ANC’s many faili...,south africa unrest anc many failings anc fail...
8978,'I didn't like it at all' - Rangnick unhappy d...,Last updated on .From the section Premier Leag...,'I didn't like it at all' - Rangnick unhappy d...,like rangnick unhappy despite cavani earning p...


## tf-idf

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, max_df=0.95)
X_tfidf = vectorizer.fit_transform(sample_data['text_cleaned']).toarray()
.shape

(8980, 19643)

In [115]:
with open('../data/sample/vectorizers/X_tfidf.pkl', 'wb') as f:
    pickle.dump(X_tfidf, f)

## Sentence transformers

In [59]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

c:\Users\jlrodriguez.andreu\AppData\Local\miniconda3\envs\unsup-classif-venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<?, ?B/s] 
README.md: 100%|██████████| 3.69k/3.69k [00:00<00:00, 3.69MB/s]
config_sentence_transformers.json: 100%|██████████| 122/122 [00:00<00:00, 120kB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:01<00:00, 63.3MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<?, ?B/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<?, ?B/s] 
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 2.35MB/s]
tokenizer_config.json: 100%|██████████| 314/314 [00:00<?, ?B/s] 
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 116MB/s]
modules.json: 100%|██████████| 229/229 [00:00<?, ?B/s] 


In [60]:
encode_transforemers = sample_data['text_cleaned'].apply(
    lambda text: model.encode(text, convert_to_numpy=True).flatten()
)

In [62]:
X_transformers = np.vstack(encode_transforemers)
X_transformers.shape

(8980, 384)

In [116]:
with open('../data/sample/vectorizers/X_transformers.pkl', 'wb') as f:
    pickle.dump(X_transformers, f)

## Bert



In [64]:
from transformers import BertModel, BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_cls_sentence(sentence):
    # Tokenize input sentence and convert to tensor
    input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True, max_length=512)])

    # Pass input through BERT model and extract embeddings for [CLS] token
    with torch.no_grad():
        outputs = model(input_ids)
        cls_embedding = outputs[0][:, 0, :]
    
    return cls_embedding.flatten()

In [65]:
cls_bert = sample_data['text_cleaned'].apply(lambda sentence: get_cls_sentence(sentence))
X_cls_bert = np.vstack(cls_bert)
X_cls_bert.shape

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(8980, 768)

In [117]:
with open('../data/sample/vectorizers/X_cls_bert.pkl', 'wb') as f:
    pickle.dump(X_cls_bert, f)

## Doc2Vec

In [72]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [104]:
index = iter(range(len(sample_data['text_cleaned'])))
tagged_data = sample_data['text_cleaned'].apply(
    lambda text: TaggedDocument(words=word_tokenize(text), tags=[next(index)])
)

In [105]:
model = Doc2Vec(vector_size=100,
                min_count=2, epochs=50)
model.build_vocab(tagged_data)
model.train(tagged_data,
            total_examples=model.corpus_count,
            epochs=model.epochs)

In [106]:
# get the document vectors
document_vectors = sample_data['text_cleaned'].apply(
    lambda text: model.infer_vector(word_tokenize(text))
)


In [107]:
X_d2v = np.vstack(document_vectors)
X_d2v.shape

(8980, 100)

In [118]:
with open('../data/sample/vectorizers/X_d2v.pkl', 'wb') as f:
    pickle.dump(X_d2v, f)

In [20]:
i = 1
print(train_data.iloc[i]['title'])
print("")
print(train_data.iloc[i]['content'])



'Nothing but problems': Shipwreck tear-down enters 5th month

Demolition of a large cargo ship along the coast of Georgia is entering its fifth month , with work to chop the ship into eight large pieces going far slower than the salvage crew anticipated SAVANNAH , Ga. -- When salvage crews began cutting apart the capsized Golden Ray , a shipwreck the size of a 70-story office building with 4,200 cars within its cargo decks , in early November they predicted the demolition could be wrapped up by New Year 's Day .
Four months later , the job remains far from finished .
Both ends of the cargo @ @ @ @ @ @ @ off by barges in a pair of giant chunks .
But roughly three-fourths of the vessel remains beached on its side off St. Simons Island on the Georgia coast , where the South Korean freighter overturned soon after leaving port Sept. 8 , 2019 .
“ It ’ s been nothing but problems out here , ” said Andy Jones , a St. Simons Island resident who heads to the wreck site in his small fishing boat 

### 1. Extraccion de tema usando el api general de hugging face

Necesitamos una lista de posibles labels, asique de momento no





### 1. Extraccion de palabras clave

https://huggingface.co/Voicelab/vlt5-base-keywords?text=Lawyer+who+took+on+tobacco+industry+now+turning+to+opioids+fight

In [61]:
# Use a pipeline as a high-level helper
from transformers import pipeline

topic_model = pipeline("text2text-generation", model="Voicelab/vlt5-base-keywords")

In [64]:
list(title)

['Singer, activist Joan Baez becomes Kennedy Center Honoree',
 "'Nothing but problems': Shipwreck tear-down enters 5th month",
 'Report: At least 13 dead in Istanbul bombings',
 'Senate Republicans Pressure Joe Biden to Withdraw Ecoterrorism-Linked Nominee',
 'The Unwelcome Return of the Real Purveyors of Violence']

In [65]:


title = list(train_data.iloc[:5]['title'])
# content = train_data.iloc[:5]['content']

result_title = topic_model(title,max_length = 20)
result_content = topic_model(content, max_length = 20)


# print(i, title)
# print('title: ', result_content)
# print('content: ', result_content)

result_title

c:\Users\jlrodriguez.andreu\AppData\Local\miniconda3\envs\unsup-classif-venv\Lib\site-packages\transformers\generation\utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'John Baez, John Baez, John Baez, John Baez, John Baez'},
 {'generated_text': 'engineering, engineering, engineering, '},
 {'generated_text': 'Istanbul bombing, Istanbul bombing'},
 {'generated_text': 'Ecoterrorism, Ecoterrorism-Linked Nominee'},
 {'generated_text': 'Violence, Violence, Violence, Violence, Violence'}]

In [74]:
i = 3
title = train_data.iloc[i]['title']
content = train_data.iloc[i]['content']
print(title)
print("")
print(content)

Senate Republicans Pressure Joe Biden to Withdraw Ecoterrorism-Linked Nominee

Several congressional Republicans are expressing their public opposition to President Joe Biden ’ s nominee to head the Bureau of Land Management ( BLM ) over her past ties to ecoterrorist activity .
Sen. John Barrasso ( R-WY ) , who is ranking member of the Committee on Energy and Natural Resources , the committee that will vote on Tracy Stone-Manning ’ s nomination ahead of a full Senate vote , said he plans to object to the nomination .
“ Tracy Stone-Manning collaborated with eco-terrorists who had booby trapped trees with metal spikes .
… She did not cooperate @ @ @ @ @ @ @ ” Barrasso said in a statement .
Sen. Jim Risch ( R-ID ) , another Energy Committee member , said she is “ unfit ” to lead the BLM and called on Biden to withdraw the nomination .
In 1993 , Stone-Manning testified against two individuals in her “ circle of friends , ” one being John P. Blount who was sentenced to 17 months in prison f

In [44]:
result = topic_model(title, max_length = 20)
result = result[0]['generated_text'].split(',')
result

['Imran Ahmad Khan',
 ' Imran Ahmad Khan',
 ' Imran Ahmad Khan',
 ' Imran Ahmad Khan']

In [19]:
result[0]

{'generated_text': 'HuggingFace, HuggingFace, HuggingFa'}

In [68]:
# otra prueba

# Use a pipeline as a high-level helper


topic_model = pipeline("token-classification", model="ml6team/keyphrase-extraction-distilbert-kptimes")

In [71]:
from transformers import (
    TokenClassificationPipeline,
    AutoModelForTokenClassification,
    AutoTokenizer,
)
from transformers.pipelines import AggregationStrategy
import numpy as np

# Define keyphrase extraction pipeline
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )

    def postprocess(self, all_outputs):
        results = super().postprocess(
            all_outputs=all_outputs,
            aggregation_strategy=AggregationStrategy.FIRST,
        )
        return np.unique([result.get("word").strip() for result in results])
    

model_name = "ml6team/keyphrase-extraction-distilbert-kptimes"
extractor = KeyphraseExtractionPipeline(model=model_name)

In [73]:
for i in range(10):

    title = train_data.iloc[i]['title']
    content = train_data.iloc[i]['content']

    result_title = extractor(title)
    result_content = extractor(content)

    # result_title = [dic['word'] for dic in result_title]
    # result_content = [dic['word'] for dic in result_content]
    print(i, title)
    print('title: ', result_content)
    print('content: ', result_content)

0 Singer, activist Joan Baez becomes Kennedy Center Honoree
title:  []
content:  []
1 'Nothing but problems': Shipwreck tear-down enters 5th month
title:  ['demolition' 'salvage']
content:  ['demolition' 'salvage']
2 Report: At least 13 dead in Istanbul bombings
title:  ['istanbul' 'turkey']
content:  ['istanbul' 'turkey']
3 Senate Republicans Pressure Joe Biden to Withdraw Ecoterrorism-Linked Nominee
title:  []
content:  []
4 The Unwelcome Return of the Real Purveyors of Violence
title:  []
content:  []
5 Imran Ahmad Khan: Wakefield MP charged with sex assault on boy
title:  []
content:  []
6 Trump: Twitter, Facebook 'a Disgrace' -- 'They Have to Be Stopped'
title:  ['donald trump' 'facebook' 'facebook.' 'social media' 'stuart']
content:  ['donald trump' 'facebook' 'facebook.' 'social media' 'stuart']
7 Australia March 4 Justice: Thousands march against sexual assault
title:  []
content:  []
8 Lawyer who took on tobacco industry now turning to opioids fight
title:  ['opioids']
content